<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_test_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install llamaapi

In [6]:
import pandas as pd
import json
from llamaapi import LlamaAPI
from google.colab import userdata
from tqdm import tqdm
import re  # Ensure 're' module is imported
import time
from google.colab import drive  # Import the drive module from google.colab
import os


# Initialize the SDK
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))

In [3]:
# mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def check_sms_legitimacy(message, model):
    request_message = f"Is this message real or fake. Just say real, fake, or undetermined. Do not give an explanation: {message}"
    while True:
        try:
            response = llama.run({
                "model": model,
                "messages": [{"role": "user", "content": request_message}]
            })

            try:
                response_json = response.json()
                used_model = response_json.get('model', 'Unknown model')
                content = response_json['choices'][0]['message']['content']
                return content, used_model
            except json.JSONDecodeError as e:
                print("Error decoding JSON:", e)
                print("Raw response text:", response.text)
                time.sleep(1)  # Wait for a second before retrying
        except Exception as e:
            print("Error during API request:", e)
            time.sleep(1)  # Wait for a second before retrying

# Specify the model you want to use
model_to_use = "mixtral-8x22b-instruct"

# Read messages from an Excel file
#input_file = os.path.abspath('/content/drive/MyDrive/Daniel hernandez/Projects/AntiSmishGPT/datasets/staged_sets/divided_mendeley_set/mendeley_set_1.xlsx')
input_file = '/content/drive/MyDrive/Daniel hernandez/Projects/AntiSmishGPT/datasets/staged_sets/divided_mendeley_set/mendeley_set_1.xlsx'
df = pd.read_excel(input_file, header=None)
df.columns = ['message']

# Process each message and get legitimacy determination
determinations = []
models = []
for i, message in enumerate(df['message'], start=1):
    determination, model = check_sms_legitimacy(message, model_to_use)
    determinations.append(determination)
    models.append(model)
    print(f"Message {i}: {determination} (Model: {model})")

# Add the determinations to the DataFrame
df['determination'] = determinations
df['model'] = models

# Get the directory of the input file
input_dir = os.path.dirname(input_file)

# Create the output file path
output_file = os.path.join(input_dir, 'processed_mendeley_set_1.xlsx')

# Save the results to a new Excel file
df.to_excel(output_file, index=False)

print("Legitimacy check completed. Results saved to:", output_file)

Message 1: Undetermined. \
Please provide more context or information to determine the validity of the statement. \
I can only provide answers based on the information given to me, and I don't have personal opinions or emotions. (Model: mixtral-8x22b-instruct)
Message 2: Undetermined. The context in which this message was sent is not provided, making it impossible to determine its authenticity. (Model: mixtral-8x22b-instruct)
Message 3: Fake (Model: mixtral-8x22b-instruct)
Message 4: Undetermined. (Model: mixtral-8x22b-instruct)
Message 5: Fake (Model: mixtral-8x22b-instruct)
Message 6: Fake. (Model: mixtral-8x22b-instruct)
Message 7: Undetermined (Model: mixtral-8x22b-instruct)
Message 8: Undetermined. (Model: mixtral-8x22b-instruct)
Message 9: Real (Model: mixtral-8x22b-instruct)
Message 10: Real. (Model: mixtral-8x22b-instruct)
Message 11: Undetermined. (Model: mixtral-8x22b-instruct)
Message 12: Undetermined. (Model: mixtral-8x22b-instruct)
Message 13: Undetermined. (Model: mixtral